In [ ]:
from koda import utils as kutils
from koda import koda
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
%matplotlib inline

koda = Koda()

for i in range(7):
    fig, axes = plt.subplots(2, 3, figsize=(10, 10))

    # Load image
    file = 'dataset/IMG_%d.jpg' % i
    img = cv2.imread(file, cv2.IMREAD_COLOR)
    koda.load(img)
    imgs = koda.pipeline.imgs    
    axes[0][1].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    axes[0][2].imshow(cv2.cvtColor(imgs['edges'], cv2.COLOR_GRAY2RGB))
    axes[0][3].imshow(cv2.cvtColor(imgs['hough_lines'], cv2.COLOR_BGR2RGB))
    axes[1][0].imshow(cv2.cvtColor(imgs['corners'], cv2.COLOR_BGR2RGB))
    axes[1][1].imshow(cv2.cvtColor(imgs['warp'], cv2.COLOR_BGR2RGB))
    axes[1][2].imshow(cv2.cvtColor(imgs['color_correction'], cv2.COLOR_BGR2RGB))


    # Plot edges detection image
    #axes[1].imshow(cv2.cvtColor(cdetector.edges_img, cv2.COLOR_GRAY2RGB))  

    # Plot Hough lines
    #im_lines = cv2.cvtColor(cdetector.edges_img.copy(), cv2.COLOR_GRAY2BGR)
    #colors = [(0,255,0),(0,0,255)]
    #for j, lines in enumerate(cdetector.hough_lines):
    #    im_lines = kutils.draw_polar_lines(im_lines, lines, colors[j])
    #axes[1].imshow(cv2.cvtColor(im_lines, cv2.COLOR_BGR2RGB))

    # Plot corners
    #im_corners = img.copy()
    #for x,y in corners:
    #     cv2.circle(im_corners, (x,y), 30, (255, 0, 0), 30)

    #axes[0].imshow(cv2.cvtColor(im_corners, cv2.COLOR_BGR2RGB))

    # Warping
    #shape = (corners.max(axis=0)[0], corners.max(axis=0)[1])
    #dst_corners = np.array([[0,0],
    #                        [0, shape[1]], 
    #                        [shape[0], 0],
    #                        [shape[0], shape[1]]])
    #M = cv2.getPerspectiveTransform(corners.astype(np.float32), dst_corners.astype(np.float32))
    #warped = cv2.warpPerspective(img, M, shape)
    
    # Plot warped image
    #axes[1].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB))
    plt.plot()

Using TensorFlow backend.
